# Prophet Application for Water Demand Forecast

## 1. Load Required Libraries

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import folium
from folium.plugins import MarkerCluster
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_squared_error as MSE
from scipy.stats import skew
import os
import sys
from glob import glob 
from calendar import day_abbr, month_abbr, mdays
from prophet import Prophet
import holidays
sys.path.append('code/')
import utils

import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning, )
%matplotlib inline

np.random.seed(42)

## 2. Load Data

### 2.1 Load Past Predictor (Independent) Variables

In [ ]:
folder_name = '특광역시'
CITY_NAME_Kor = '서울특별시'
CITY_NAME_Eng = 'Seoul'
predictor_raw = pd.read_excel("data/cities_predictor_variables.xlsx", sheet_name=CITY_NAME_Kor)
col_name = predictor_raw.iloc[2].values
predictor_raw1 = predictor_raw.iloc[3:, :29] # header 및 일단위자료 삭제
predictor_raw1.columns = col_name[:29]
predictor_raw1_len = predictor_raw1['연도'].isna().argmax()
predictor_raw1 = predictor_raw1[:predictor_raw1_len]
predictor_raw1.index = pd.to_datetime(predictor_raw1['연도'][:predictor_raw1_len].astype(str) + '-' + predictor_raw1['월'][:predictor_raw1_len].astype(str))
predictor_raw1.rename_axis(columns='', inplace=True)
predictor_raw1.drop(columns=['연도', '월'], inplace=True)
predictor_raw1.index.name = "date"
predictor_raw1 = predictor_raw1.astype(float)
predictor_raw1.columns = ['Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 'Male_Female_Ratio', 'Population_aging_Ratio', 
                'Power_usage', 'Num_of_Business', 'Business_above_100', 'complex_area', 'annual_household_income',
                'High_School_Graduate_num', 'High_School_Graduate_ratio', 'personal_expense', 'benefits_vs_personal_expense', 'employment_ratio', 'employment_insurance_ratio', 'vulnerable_class', 'vulnerable_class_ratio',
                'Temp', 'Rainfall', 'Humidity', 'Solar_radiation', 'Ground_Temp', 'Wind', 'Pressure']
predictor_f = predictor_raw1.loc["2017-01-01":"2021-12-01"]
predictor_f.head()

### 2.2 Load Past Response (Dependent) Variable: 용수공급량 ("월별 공급량/월별 일수"를 통해 m3/day으로 평균하여 표현)

In [ ]:
import pandas as pd

water_supply_raw = pd.read_excel("data/water_supply(17-21).xlsx", header=1)
water_supply_raw["ds"] = water_supply_raw["지자체명"].values
water_supply_raw = water_supply_raw.iloc[1:, 1:]
water_supply_raw.head()

### 2.3 Load Future Predictor (Independent) Variables

In [ ]:
future_factor = pd.read_excel("data/cities_future_predictor_variables_locality.xlsx", sheet_name=CITY_NAME_Kor)
future_factor_1 = future_factor.iloc[3:19, :8] # header 및 일단위자료 삭제
future_factor_1.columns = future_factor.iloc[2, :8] # set column names
future_factor_1.index = pd.to_datetime(future_factor_1["연도"], format='%Y')
future_factor_1 = future_factor_1.resample('MS').ffill()
future_factor_1.index.name = "date"
future_factors1 = future_factor_1[future_factor_1.columns[1:]]
#future_factors1.columns = ['총인구수', '세대수', '세대당 인구', '남자 인구수', '여자 인구수', '남여 비율', '고령화비율']
future_factors1.columns = ['Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 
                           'Male_Female_Ratio', 'Population_aging_Ratio']
future_factors1 = future_factors1.astype(float)
future_factors1

### 2.4 Load Future Response (Dependent) Variable: 용수공급량 ("월별 공급량/월별 일수"를 통해 m3/day으로 평균하여 표현)

In [ ]:
future_factor_3 = future_factor.iloc[135:316, 22:28] # header 및 일단위자료 삭제
future_factor_3.columns = future_factor.iloc[2, 22:28] # set column names
future_factor_3.index = pd.to_datetime(future_factor_1.index)
future_factor_3.index.name = "date"
future_factors2 = future_factor_3[['월합강수량(mm)', '습도\n평균상대습도(%)', '평균온도(°C)']]
#future_factors2.columns = ['월강수량', '습도', '기온']
future_factors2.columns = ['Rainfall', 'Humidity', 'Temp']
future_factors2 = future_factors2.astype(float)
future_factors2

### 2.5 Merge Every Dataset

In [ ]:
future_factors1_1 = future_factors1.loc["2022-01-01":"2035-01-01"]
future_factors2_1 = future_factors2.loc["2022-01-01":"2035-01-01"]
future_factor = pd.concat([future_factors1_1, future_factors2_1], axis=1)
predictor_f["y"] = water_supply_raw[CITY_NAME_Kor].values
total_data_df1 = pd.concat([predictor_f, future_factor], axis=0)

total_data_df = total_data_df1[['y', 'Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 
                               'Male_Female_Ratio', 'Population_aging_Ratio', 'Rainfall', 'Humidity', 'Temp']]
total_data_df

## 3. Application of Prophet

### 3.1 Univariate Analysis using past water supply

#### 3.1.1 Prepare Training and Test data

In [ ]:
data_train = total_data_df.loc["2017-01-01":"2020-12-01"][['y']]
data_train['ds'] = data_train.index
#data_train.index = range(0, len(data_train))
data_test = total_data_df.loc["2021-01-01":"2021-12-01"][['y']]
data_test['ds'] = data_test.index

#### 3.1.2 Training and Testing using Multiplicative Decomposition

In [ ]:
m1 = Prophet(mcmc_samples=300, changepoint_prior_scale=0.9, seasonality_mode='multiplicative', yearly_seasonality=10)
m1.fit(data_train)
future1_1 = m1.make_future_dataframe(periods=len(data_test), freq='MS')
verification1 = m1.predict(future1_1)
MAE_value = MAE(verification1['yhat'][-12:], data_test['y'])
R2_value = R2(verification1['yhat'][-12:], data_test['y'])
RMSE_value = MSE(verification1['yhat'][-12:], data_test['y'])**0.5
Correlation = np.corrcoef(verification1['yhat'][-12:], data_test['y'])[0,1]
print("MAE="+ str(MAE_value), "R2="+ str(R2_value), "RMSE="+ str(RMSE_value), "Correlation="+ str(Correlation))
verif_fig = m1.plot(verification1)

#### 3.1.3 verifing train and test set

In [ ]:
verif_d1 = utils.make_verif(verification1, data_train, data_test)
f =  utils.plot_verif(verif_d1)

In [ ]:
utils.plot_joint_plot(verif_d1.loc[:'2020-12-01',:], title='train set', fname='train_set_joint_plot_climate')

In [ ]:
utils.plot_joint_plot(verif_d1.loc['2021-01-01':,:], title='test set', fname='test_set_joint_plot_no_climate')

In [ ]:
residuals1 = verif_d1.loc['2021-01-01':,'yhat'] - verif_d1.loc['2021-01-01':,'y']
f, ax = plt.subplots(figsize=(8,8))
sns.distplot(residuals1, ax=ax, color='0.4')
ax.grid(ls=':')
ax.set_xlabel('residuals', fontsize=15)
ax.set_ylabel("normalised frequency", fontsize=15)
ax.grid(ls=':')

[l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
[l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()];

ax.axvline(0, color='0.4')
ax.set_title('Residuals distribution (test set)', fontsize=17)

ax.text(0.05, 0.85, "Skewness = {:+4.2f}\nMedian = {:+4.2f}\nMean = {:+4.2f}".\
        format(skew(residuals1), residuals1.median(), residuals1.mean()), \
        fontsize=14, transform=ax.transAxes)

for ext in ['png','jpeg','pdf']: 
    f.savefig(f'residuals_distribution_test_0.{ext}', dpi=200)

In [ ]:
f = m1.plot_components(verification1)

#### 3.1.4 forcast water demand until 2035 year

In [ ]:
data_future1 = total_data_df.loc["2021-01-01":"2035-01-01"][['y']]
data_future1['ds'] = data_future1.index
future1_2 = m1.make_future_dataframe(periods=len(data_future1), freq='MS')
forecast1 = m1.predict(future1_2)
forecast_fig_1 = m1.plot(forecast1)
ax = forecast_fig_1.gca()
ax.set_ylim(2500000, 3600000)

#### 3.1.5 Training and Testing using additive Decomposition

In [ ]:
m2 = Prophet(mcmc_samples=300, changepoint_prior_scale=0.8, seasonality_mode='additive', yearly_seasonality=10)
m2.fit(data_train)
future2_1 = m2.make_future_dataframe(periods=len(data_test), freq='MS')
verification2 = m2.predict(future2_1)
MAE_value = MAE(verification2['yhat'][-12:], data_test['y'])
R2_value = R2(verification2['yhat'][-12:], data_test['y'])
RMSE_value = MSE(verification2['yhat'][-12:], data_test['y'])**0.5
Correlation = np.corrcoef(verification2['yhat'][-12:], data_test['y'])[0,1]
print("MAE="+ str(MAE_value), "R2="+ str(R2_value), "RMSE="+ str(RMSE_value), "Correlation="+ str(Correlation))
verif_fig = m2.plot(verification2)
plt.show()

In [ ]:
verif_d2 = utils.make_verif(verification2, data_train, data_test)
f =  utils.plot_verif(verif_d2)

In [ ]:
utils.plot_joint_plot(verif_d2.loc['2021-01-01':,:], title='test set', fname='test_set_joint_plot_no_climate')

In [ ]:
residuals2 = verif_d2.loc['2021-01-01':,'yhat'] - verif_d2.loc['2021-01-01':,'y']
f, ax = plt.subplots(figsize=(8,8))
sns.distplot(residuals2, ax=ax, color='0.4')
ax.grid(ls=':')
ax.set_xlabel('residuals', fontsize=15)
ax.set_ylabel("normalised frequency", fontsize=15)
ax.grid(ls=':')

[l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
[l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()];

ax.axvline(0, color='0.4')
ax.set_title('Residuals distribution (test set)', fontsize=17)

ax.text(0.05, 0.85, "Skewness = {:+4.2f}\nMedian = {:+4.2f}\nMean = {:+4.2f}".\
        format(skew(residuals2), residuals2.median(), residuals2.mean()), \
        fontsize=14, transform=ax.transAxes)

for ext in ['png','jpeg','pdf']: 
    f.savefig(f'residuals_distribution_test_set_1.{ext}', dpi=200)

In [ ]:
f = m2.plot_components(verification2)

In [ ]:
data_future2 = total_data_df.loc["2021-01-01":"2035-01-01"][['y']]
data_future2['ds'] = data_future2.index
future2_2 = m2.make_future_dataframe(periods=len(data_future2), freq='MS')
forecast2 = m2.predict(future2_2)
forecast_fig_2 = m2.plot(forecast2)
ax = forecast_fig_2.gca()
ax.set_ylim(2500000, 3700000)

### 3.2 Multivariate Analysis for water demand forecast

#### 3.2.1 Create m3 object using Prophet and add regressor to use multivariate

In [ ]:
m3 = Prophet(mcmc_samples=300, changepoint_prior_scale=0.9, seasonality_mode='multiplicative', yearly_seasonality=10)

In [ ]:
m3.add_regressor('Rainfall', prior_scale=0.2, mode='multiplicative')
m3.add_regressor('Humidity', prior_scale=0.2, mode='multiplicative')
m3.add_regressor('Temp', prior_scale=0.2, mode='multiplicative')
m3.add_regressor('Total_Population', prior_scale=0.2, mode='multiplicative')
m3.add_regressor('Households', prior_scale=0.2, mode='multiplicative')
m3.add_regressor('Population_per_Households', prior_scale=0.2, mode='multiplicative')

In [ ]:
total_data_df['ds'] = total_data_df.index
data_train = total_data_df[['ds', 'y', 'Rainfall', 'Humidity', 'Temp', 'Total_Population', 'Households', 'Population_per_Households']].loc["2017-01-01":"2020-12-01"]
data_test = total_data_df[['ds', 'y', 'Rainfall', 'Humidity', 'Temp', 'Total_Population', 'Households', 'Population_per_Households']].loc["2021-01-01":"2021-12-01"]

In [ ]:
m3.fit(data_train)

In [ ]:
future3_1 = m3.make_future_dataframe(periods=len(data_test), freq='MS')
futures3_1 = utils.add_regressor_to_future(future3_1, [total_data_df['Rainfall'], total_data_df['Humidity'], total_data_df['Temp'],
                                         total_data_df['Total_Population'], total_data_df['Households'], total_data_df['Population_per_Households']])
verification3 = m3.predict(futures3_1)
MAE_value = MAE(verification3['yhat'][-12:], data_test['y'])
R2_value = R2(verification3['yhat'][-12:], data_test['y'])
RMSE_value = MSE(verification3['yhat'][-12:], data_test['y'])**0.5
Correlation = np.corrcoef(verification3['yhat'][-12:], data_test['y'])[0,1]
print(MAE_value, R2_value, RMSE_value, Correlation)
verif_fig = m3.plot(verification3)
plt.show()

In [ ]:
verif_d3 = utils.make_verif(verification3, data_train, data_test)
f =  utils.plot_verif(verif_d3)

In [ ]:
utils.plot_joint_plot(verif_d3.loc[:'2020-12-01',:], title='train set', fname='train_set_joint_plot_climate')

In [ ]:
utils.plot_joint_plot(verif_d3.loc['2021-01-01':,:], title='test set', fname='test_set_joint_plot_no_climate')

In [ ]:
residuals3 = verif_d3.loc['2021-01-01':,'yhat'] - verif_d3.loc['2021-01-01':,'y']
f, ax = plt.subplots(figsize=(8,8))
sns.distplot(residuals3, ax=ax, color='0.4')
ax.grid(ls=':')
ax.set_xlabel('residuals', fontsize=15)
ax.set_ylabel("normalised frequency", fontsize=15)
ax.grid(ls=':')

[l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
[l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()];

ax.axvline(0, color='0.4')
ax.set_title('Residuals distribution (test set)', fontsize=17)

ax.text(0.05, 0.85, "Skewness = {:+4.2f}\nMedian = {:+4.2f}\nMean = {:+4.2f}".\
        format(skew(residuals3), residuals3.median(), residuals3.mean()), \
        fontsize=14, transform=ax.transAxes)

for ext in ['png','jpeg','pdf']: 
    f.savefig(f'residuals_distribution_test_set_2.{ext}', dpi=200)

In [ ]:
f = m3.plot_components(verification3)

In [ ]:
data_future3 = total_data_df[['ds', 'y', 'Rainfall', 'Humidity', 'Temp', 'Total_Population', 'Households', 'Population_per_Households']].loc["2021-01-01":"2035-01-01"]
future3_2 = m3.make_future_dataframe(periods=len(data_future3), freq='MS')
futures3_2 = utils.add_regressor_to_future(future3_2, [total_data_df['Rainfall'], total_data_df['Humidity'], total_data_df['Temp'],
                                                     total_data_df['Total_Population'], total_data_df['Households'], total_data_df['Population_per_Households']])
forecast3 = m3.predict(futures3_2)
forecast_fig_3 = m3.plot(forecast3)
ax = forecast_fig_3.gca()
ax.set_ylim(2500000, 3800000)